In [196]:
#importacion de librerias
import pandas as pd
import numpy as np
import ast
#para el inicio sin datos
import random

#para el modelo
from sklearn.metrics.pairwise import cosine_similarity

# Apertura de archivos

In [194]:
businessy = pd.read_csv('C_Business.csv')
businessry = pd.read_csv('C_Reviews.csv')
users = pd.read_csv('Users.csv')


# Creación de categorias para el modelo
Se crean varias categorias del analisis del texto de los comentarios, para entrenar el modelo de machine learning con mayor información

## Categoria interaccion
Se usa para medir la interacion de los demas usuario de yelp con cada usuario en particular, no se incluye rate. ya que se incluira el rate pero por negocio en particular, a travez del rate que esta en los reviews.

In [100]:
#Crea la categoria interaccion de la suma de columnas de varias colunmas de interaccion entre el numero de comentarios por usuario
usery['interaccion'] = ( usery['useful'] + usery['funny'] + usery['cool'] + usery['fans'] / usery['review_count'] )


#Creacion de la categoria binaria 
#valor de la mediana
mediana_interaccion = usery['interaccion'].median()
#Se inicializa la columna con el valor 0
usery['cat_interaccion'] = 0 
#Agrega 1 a la nueva columna cuando interaccion es mayor que su mediana
usery.loc[usery['interaccion'] > mediana_interaccion*(4/5), 'cat_interaccion'] = 1
#borra la columna interaccion
usery.drop(columns={'interaccion'}, axis=1, inplace=True)

## Categoria elite
Se hace un analisis de las palabras de los reviews, con palabras que le dan cieta categoria a los salones.


In [101]:
#lista de palabras para la categoria elite
lista_palabras = ["great experience",  "great",  "happy",  "trust",  "designs",  "exactly what i wanted", "exactly", "wanted", "extremely talented", "talented",
"amazing work", " amazing", "immediatel", "extremely nice", "nice", "very precisely", "precisely", "fastest", "spacious salon", "spacious", "wine", "chocolate",
"soda", "friendly", "fantastic stylist", "fantastic", "great pedicurist", "great pedicure", "perfect", "absolutely amazing", "amazing","highly recommend",
"very clean", "clean", "very relaxing", "relaxing", "thank", "great experience", "was awesome", "awesome", "unique", "place unique", "great", "customer", "service"
"luxurious", "luxury", "spa", "spas", "genuinely", "helpful", "sweet", "love", "loved", "talented", "excellent", "best", "originally", "original", "Organized" ]

#pasa las palabras de la lista a minusculas
lista_palabras = [palabra.lower() for palabra in lista_palabras]

#Hace la comparacion de cada palabara de la lista en los textos de los comentarios,
#cuenta cuantas veces aparece una palabra de la lista por comentario, se guarda en una lista y se agrega al daraframe
lista=[]
for l in range(0,len(businessry)):
    count = 0
    for i,k in enumerate(lista_palabras):

         if k in list(businessry['text'].iloc[l].lower().replace(',','').replace('.','').split()):
            count =+1
    lista.append(count)   
businessry['elite'] = pd.DataFrame(lista)        

#crea la columna final cat_elite
businessry['cat_elite'] = 0 
#Agrega 1 a la nueva columna cuando limpieza es mayor que 1
businessry.loc[businessry['elite'] >= 1, 'cat_elite'] = 1
#borra la columna limpieza
businessry.drop(columns={'elite'}, axis=1, inplace=True)

## Categoria limpieza
analisa palabras que estan relacionadas con la limpieza del salon.

In [102]:
#lista de palabras para la categoria Limpieza
lista_palabras = ["Cleanliness", "Sanitary", "Hygienic", "Organized", "Tidy", "Neat", "Sterile", "Spotless", "Immaculate", "Orderly","Clean",
"Well maintained", "Squeaky clean", "Clutter free", "Pristine", "Fresh", "Spacious", "Meticulous", "Well groomed","Freshly",
"Well kept", "Structured", "Efficient", "Methodical", "Systematic", "Arranged", "Presentable", "Gleaming", "Polished",
"Crystalline", "Aseptic", "Uncluttered", "Sleek", "Trim", "Groomed", "Hygienized", "Disinfected", "Tended", "Spartan",
"Trimmed", "Kempt", "Slick", "Unblemished", "Clear", "Unsoiled", "Unspoiled", "Unblemished", "Aseptic", "Trimmed", "Shining"]

#pasa las palabras de la lista a minusculas
lista_palabras = [palabra.lower() for palabra in lista_palabras]

#Hace la comparacion de cada palabara de la lista en los textos de los comentarios,
#cuenta cuantas veces aparece una palabra de la lista por comentario, se guarda en una lista y se agrega al daraframe
lista=[]
for l in range(0,len(businessry)):
    count = 0
    for i,k in enumerate(lista_palabras):

         if k in list(businessry['text'].iloc[l].lower().replace(',','').replace('.','').split()):
            count =+1
    lista.append(count)   
businessry['limpieza'] = pd.DataFrame(lista)        

#crea la columna final cat_limpieza
businessry['cat_limpieza'] = 0 
#Agrega 1 a la nueva columna cuando limpieza es mayor que 1
businessry.loc[businessry['limpieza'] >= 1, 'cat_limpieza'] = 1
#borra la columna limpieza
businessry.drop(columns={'limpieza'}, axis=1, inplace=True)

## Categoria ambiente
analisa como esta el ambiente emocional del salon, como es el trato a los usuarios.

In [103]:
#lista de palabras para la categoria Limpieza
lista_palabras = ["Courteous", "Polite", "Friendly", "Welcoming", "Attentive", "Respectful", "Helpful", "Warm", "Cordial",
"Approachable", "Customer focused", "Personable", "Service oriented", "Professional", "Amiable", "Cheerful",
"Considerate", "Kind", "Patient", "Understanding", "Responsive", "Empathetic", "Courageous", "Generous",
"Gracious", "Grateful", "Hospitable", "Inviting", "Gratifying", "Pleasant", "Thoughtful", "Customer centric",
"Gratifying", "Obliging", "Sociable", "Affable", "Gentle", "Good humored", "Grateful", "Amicable", "Engaging",
"Enthusiastic", "Good-natured", "Grateful", "Sociable", "Amicable", "Charming", "Compassionate", "Cooperative"]


#pasa las palabras de la lista a minusculas
lista_palabras = [palabra.lower() for palabra in lista_palabras]

#Hace la comparacion de cada palabara de la lista en los textos de los comentarios,
#cuenta cuantas veces aparece una palabra de la lista por comentario, se guarda en una lista y se agrega al daraframe
lista=[]
for l in range(0,len(businessry)):
    count = 0
    for i,k in enumerate(lista_palabras):

         if k in list(businessry['text'].iloc[l].lower().replace(',','').replace('.','').split()):
            count =+1
    lista.append(count)   
businessry['ambiente'] = pd.DataFrame(lista)        

#crea la columna final cat_limpieza
businessry['cat_ambiente'] = 0 
#Agrega 1 a la nueva columna cuando limpieza es mayor que 1
businessry.loc[businessry['ambiente'] >= 1, 'cat_ambiente'] = 1
#borra la columna ambiente
businessry.drop(columns={'ambiente'}, axis=1, inplace=True)

## Categoria promociones
analisa si el sition tiene descuentos, proposiones o cupones de descuentos.

In [104]:
#lista de palabras para la categoria promciones
lista_palabras = ["Affordable", "Reasonable", "Competitive", "Budget", "friendly", "Cost effective", "Economical", "Inexpensive",
"Attractive","pricing", "Value for money", "Discounted", "Special offers", "Savings", "Cost" ,"efficient",
"Reasonably priced", "Economically priced", "Low cost", "Wallet friendly", "Bargain", "Discounts",
"Promotional prices", "Saver deals", "Reduced rates", "Money saving", "Affordable rates","low","cost" 
"Economically priced", "Discounted", "offers", "Competitive prices", "Pocket",  "Inexpensive deals",
"Affordable","options", "Sale", "Cut rate", "Low priced", "Thrifty", "Reasonably valued", "Marked down",
"On", "sale", "Cost","conscious", "Attractive discounts", "Economical choices", "Savings","opportunities",
"Discounted","packages", "Special promotions", "Budget conscious", "Discounted rates", "Affordable","packages",
"Promo", "Reduced", "cost", "Economic", "options", "Money", "saving", "deals", "Discounted specials"]


#pasa las palabras de la lista a minusculas
lista_palabras = [palabra.lower() for palabra in lista_palabras]

#Hace la comparacion de cada palabara de la lista en los textos de los comentarios,
#cuenta cuantas veces aparece una palabra de la lista por comentario, se guarda en una lista y se agrega al daraframe
lista=[]
for l in range(0,len(businessry)):
    count = 0
    for i,k in enumerate(lista_palabras):

         if k in list(businessry['text'].iloc[l].lower().replace(',','').replace('.','').split()):
            count =+1
    lista.append(count)   
businessry['promociones'] = pd.DataFrame(lista)        

#crea la columna final cat_limpieza
businessry['cat_promociones'] = 0 
#Agrega 1 a la nueva columna cuando limpieza es mayor que 1
businessry.loc[businessry['promociones'] >= 1, 'cat_promociones'] = 1
#borra la columna promociones
businessry.drop(columns={'promociones'}, axis=1, inplace=True)

## Categoria extra
Analisa cosas que no estan en las categorias anteriores, como si te ofrecen servicios extras como spa, refrigerios, corte de cabello y servicios. especificos

In [105]:
#lista de palabras para la categoria extra
lista_palabras = ["Haircut", "Hairstyling", "coloring", "Highlights", "Lowlights", "Balayage", "Ombre", "extensions",
 "straightening", "perm", "Keratin", "smoothing", "Blowout", "Updo", "Braiding", "braids", "hairstyles", "Men", "Women", 
 "Children", "haircut","Pixie", "Bob", "Shag", "Bangs", "Curls", "Curly", "conditioning", "massage", "Deep", "repair", 
 "glossing", "glazing", "treatment", "spa", "rejuvenation", "health", "consultation", "care" ,"tips", "hydration", "oil", "mask", 
"trimming", "shaping", "Beard", "grooming", "Mustache",  "Children's hair styling", "Bridal hair styling", "consultation", "wine",
"hair", "spa" "Manicure", "Pedicure", "Acrylic", "Gel", "extensions", "art", "design", "French","Shellac", "SNS", "Dip" ,
"powder", "repair", "removal", "shaping", "Cuticle", "care","Paraffin", "wax", "strengthening", "Nails", "polish",
"buffing", "Foot"," massage", "Hand", "Callus"," removal", "scrub", "Moisturizing" , "Hot oil treatment", "Nail spa", 
"Nail rejuvenation", "Nail health consultation", "Nail care tips", "Nail conditioning", "hydration", 
"Cuticle oil", "Nail mask", "Gel removal", "filing", "varnish", "Chrome", "Ombre",
"Matte", "Glitter", "Natural", "Quick dry options", "wraps", "parking",
"embellishments", "gems", "decals", "stickers", "color" ,"options", "Signature"]



#pasa las palabras de la lista a minusculas
lista_palabras = [palabra.lower() for palabra in lista_palabras]

#Hace la comparacion de cada palabara de la lista en los textos de los comentarios,
#cuenta cuantas veces aparece una palabra de la lista por comentario, se guarda en una lista y se agrega al daraframe
lista=[]
for l in range(0,len(businessry)):
    count = 0
    for i,k in enumerate(lista_palabras):

         if k in list(businessry['text'].iloc[l].lower().replace(',','').replace('.','').split()):
            count =+1
    lista.append(count)   
businessry['extra'] = pd.DataFrame(lista)        

#crea la columna final cat_limpieza
businessry['cat_extra'] = 0 
#Agrega 1 a la nueva columna cuando limpieza es mayor que 1
businessry.loc[businessry['extra'] >= 1, 'cat_extra'] = 1
#borra la columna extra
businessry.drop(columns={'extra'}, axis=1, inplace=True)

# Creación de dataframe para entrenar el modelo
Construccion del dataframe para entrenar el modelo, para generar la matriz y usar la similitud de los cosenos

In [106]:
businessry.columns

Index(['user_id', 'business_id', 'review_stars', 'text', 'date', 'year',
       'source', 'cat_elite', 'cat_limpieza', 'cat_ambiente',
       'cat_promociones', 'cat_extra'],
      dtype='object')

In [107]:
businessry.shape

(15591, 12)

In [108]:
df_temp = businessry[['user_id', 'business_id', 'review_stars','cat_elite', 'cat_limpieza', 'cat_ambiente',
       'cat_promociones', 'cat_extra']]

In [109]:
usery.shape

(11353, 10)

In [110]:
df_final=df_temp.merge(usery, how='left', on='user_id')

In [111]:
df_final.columns

Index(['user_id', 'business_id', 'review_stars', 'cat_elite', 'cat_limpieza',
       'cat_ambiente', 'cat_promociones', 'cat_extra', 'name', 'review_count',
       'yelping_since', 'useful', 'funny', 'cool', 'fans', 'average_stars',
       'cat_interaccion'],
      dtype='object')

In [112]:
df_final.drop([ 'name', 'review_count', 'yelping_since',
       'useful', 'funny', 'cool', 'fans', 'average_stars'], axis=1, inplace=True)

In [113]:
df_final.columns

Index(['user_id', 'business_id', 'review_stars', 'cat_elite', 'cat_limpieza',
       'cat_ambiente', 'cat_promociones', 'cat_extra', 'cat_interaccion'],
      dtype='object')

In [114]:
df_final.shape

(15591, 9)

Para dejar el promedio de las categorias y el rate por negocio

In [115]:
#quita la columna user_id
df_final.drop([ 'user_id'], axis=1, inplace=True)

#saca los valores unicos de la columna business_id en un arreglo
unicos = df_final['business_id'].unique()

In [116]:
#prueba por un error en el codigo
#df_final.dropna(inplace=True)
#df_final['cat_interaccion'] = df_final['cat_interaccion'].astype(int)

In [ ]:
#saca un promedio de todas las filas de de los valores unicos para todas la columnas
lista_stars=[];lista_elite=[];lista_limpieza=[];lista_ambiente=[]
lista_promociones=[];lista_extra=[];lista_interaccion=[];lista_business=[]
for i in range(0,len(unicos)):
    df = df_final[df_final['business_id']==unicos[i]]
    lista_business.append(unicos[i])
    lista_stars.append(int(df['review_stars'].mean().round()))
    lista_elite.append(int(df['cat_elite'].mean().round()))
    lista_limpieza.append(int(df['cat_limpieza'].mean().round()))
    lista_ambiente.append(int(df['cat_ambiente'].mean().round()))
    lista_promociones.append(int(df['cat_promociones'].mean().round()))
    lista_extra.append(int(df['cat_extra'].mean().round()))
    lista_interaccion.append(int(df['cat_interaccion'].mean().round()))

In [ ]:
#Crea un data frame a partir de las listas
df_ml_final = pd.DataFrame({'business_id': lista_business, 'review_stars': lista_stars, 'cat_elite':lista_elite,'cat_limpieza':lista_limpieza,
                            'cat_ambiente':lista_ambiente,'cat_promociones':lista_promociones,'cat_extra':lista_extra,'cat_interaccion':lista_interaccion})

In [ ]:
df_ml_final.to_csv('df.csv', index=False)

# Creación de Modelo 

Se decidio por optimizacion de recursos usar un modelo sencillo para user_id - business_id y luego un business_id - business_id, usando las categoria generadas anteriormente, se construyo un modelo que usa la similitud del coseno.




## Creación de Modelo user_id - business_id

Para efectos esteticos se genero un correo unico por usuario, por lo que se recomendara por cada correo pasado un business_id.
Si no esta en la base de datos, se notificara que no esta y de igual manera se le dara una recomendacion

Se construyó a mano un metodo sencillo para tomar un correo y dar una recomendación de business_id, incluso si el corre no se encuentra en los datos

In [241]:
#por la poca cantidad de informacion en el modelo, suele repetir ciertas recomendaciones

#correo ='jose@gmail' # no esta en la base de datos
#correo ='jose@gmail.com' # no esta en la base de datos
#correo ='tera3@gmail.com' # esta en la base de datos 815
#correo ='virginia11@gmail.com' # esta en la base de datos 815
#correo ='beth49@gmail.com' # esta en la base de datos 230
#correo ='jodi90@gmail.com' # esta en la base de datos 758
#correo ='maria115@gmail.com' # esta en la base de datos 229
#correo ='happiness152@gmail.com' # esta en la base de datos 97
correo ='katie202@gmail.com' # esta en la base de datos 12


if correo.find("@") == -1 or correo.find(".com") == -1 or correo.find(" ") != -1:
          print("Coloque un correo válido.")
else:          

     businessry_ord=businessry.sort_values(by='review_stars', ascending=False)
     businessry_ord_unic=businessry.drop_duplicates(subset=['business_id'])

     try:
        usuario = users[users['correo']==correo].iloc[0][0]
     except IndexError:
        business_id_recomendado = businessry_ord_unic.iloc[0][1]  


     try:
        df_usuario = businessry_ord[businessry_ord['user_id']==usuario].sort_values(by='review_stars', ascending=False)
     except:
       print('No estas en nuestra base de datos, te daremos una recomendación de igual forma.')
       pass   
          
     n = random.randint(0, 100)

     try:
       business_id_recomendado = df_usuario.iloc[0][1]
     except:
       business_id_recomendado = businessry_ord_unic.iloc[n][1]

business_id_recomendado

'z-s_l9SLZiqhAArcdkYOaw'

## Creacion Modelo business_id - business_id

Se utilizó las categorías generadas anteriormente por business_id para generar un business_id recomendado

Este modelo conecta con el anterior, la respuesta del anterior se usa aca como entrada

In [242]:
df = pd.read_csv('df.csv')
#carga modelos
df_modelo_entrenamiento = df[[ 'cat_elite',	'cat_limpieza',	'cat_ambiente',	'cat_promociones',	'cat_extra',	'cat_interaccion']]

cosine_sim = cosine_similarity(df_modelo_entrenamiento, df_modelo_entrenamiento)


12

In [192]:
#business_id_recomendado
#business_id_recomendado= '16bxqGvRmT3yUdNdluuJXg'
#business_id_recomendado= '14wug4g_h4K-8zqxGIQ7gw'
#business_id_recomendado= '-hWqt93mXwiPUmS12suf6w'

In [244]:
idx = df[df['business_id'] == business_id_recomendado].index[0]
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores = sim_scores[2:7]
business_indices = [i[0] for i in sim_scores]
lista_business_id=[]
for i,k in enumerate(business_indices):
    lista_business_id.append(df.iloc[k][0])
lista_business_id
#lista_business_id = list(df['business_id'].iloc[business_indices])
#df_temp = df['business_id'].iloc[business_indices].reset_index

lista_nombre=[]; lista_direccion=[]
 
      
#           #Revisar bien al usar el codigo final, porque los diccionarios no guardan orden
for i,k in enumerate(lista_business_id):
               if lista_business_id[i] != business_id_recomendado:

                  lista_nombre.append( businessy[businessy['business_id']==lista_business_id[i]].iloc[0][2]) 
                  lista_direccion.append(businessy[businessy['business_id']==lista_business_id[i]].iloc[0][3])  
    

In [246]:
print('Nombres de locales:', lista_nombre)

Nombres de locales: ['Lily Nails and Spa', 'The Pamper Stop', 'Nova Nails Spa', 'Luna Luxe Nail Loft', 'Prestige Natural Nail Bar']
